In [2]:
import requests
from time import sleep

from tqdm import tqdm
import pymongo
from pymongo import MongoClient
import gensim

import logging
logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)
logging.root.level = logging.INFO

In [3]:
import sys
sys.path.append('../')
from src import settings
client = MongoClient(settings.MONGODB_SETTINGS['host'])
db = client[settings.MONGODB_SETTINGS['db']]
col = db[settings.MONGODB_SETTINGS['collection']]

In [4]:
posts = col.find()

In [5]:
col.count_documents({})

177

In [6]:
import joblib
doc_topic_dist = joblib.load(
        settings.PATH_DOC_TOPIC_DIST
    )

In [7]:
def make_sentences():
    for post in col.find():
        yield post['pp_content'] # lấy thuộc tính pp_content của từng document trong post collection
def make_texts_corpus(sentences):
    for sentence in sentences:
        yield simple_preprocess(sentence, deacc=True) # tách chữ cái của từng pp_content
#         yield [word for word in sentence.split()]
from gensim.models.coherencemodel import CoherenceModel
from gensim.utils import simple_preprocess

In [8]:
corpus = gensim.corpora.MmCorpus('../models/corpus.mm')
id2word = gensim.corpora.Dictionary.load(settings.PATH_DICTIONARY)

INFO : loaded corpus index from ../models/corpus.mm.index
INFO : initializing cython corpus reader from ../models/corpus.mm
INFO : accepted corpus with 166 documents, 4138 features, 35081 non-zero entries
INFO : loading Dictionary object from ../models/id2word.dictionary
INFO : Dictionary lifecycle event {'fname': '../models/id2word.dictionary', 'datetime': '2023-01-01T08:30:14.082846', 'gensim': '4.2.0', 'python': '3.9.6 (tags/v3.9.6:db3ff76, Jun 28 2021, 15:26:21) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'loaded'}


In [9]:
lda_model = gensim.models.ldamodel.LdaModel(corpus, id2word=id2word, num_topics=57, passes=10,
            chunksize=67, random_state=46, alpha=1e-2, eta=0.5e-2,
            minimum_probability=0.0, per_word_topics=False)

INFO : using serial LDA version on this node
INFO : running online (multi-pass) LDA training, 57 topics, 10 passes over the supplied corpus of 166 documents, updating model once every 67 documents, evaluating perplexity every 166 documents, iterating 50x with a convergence threshold of 0.001000
INFO : PROGRESS: pass 0, at document #67/166
INFO : merging changes from 67 documents into a model of 166 documents
INFO : topic #27 (0.010): 0.025*"agile" + 0.020*"marketing" + 0.010*"stories" + 0.010*"calo" + 0.009*"user" + 0.009*"san_pham" + 0.009*"da" + 0.009*"khach_hang" + 0.008*"sprint" + 0.007*"can"
INFO : topic #11 (0.010): 0.014*"agile" + 0.012*"ta" + 0.012*"moi_truong" + 0.010*"du_an" + 0.010*"thong_minh" + 0.007*"khach_hang" + 0.006*"quan_ly" + 0.006*"da" + 0.006*"he_thong" + 0.006*"bao_ve"
INFO : topic #15 (0.010): 0.019*"khoi_nghiep" + 0.019*"kinh_doanh" + 0.018*"da" + 0.011*"kinh_te" + 0.009*"đau_tu" + 0.008*"the_gioi" + 0.008*"huong" + 0.007*"cong_ty" + 0.006*"lan" + 0.006*"thi_tr

INFO : topic #17 (0.010): 0.195*"da" + 0.057*"san_pham" + 0.048*"lan" + 0.038*"duong" + 0.032*"cham_soc" + 0.030*"dau" + 0.022*"sach" + 0.022*"am" + 0.018*"mat" + 0.017*"kho"
INFO : topic diff=8.324760, rho=0.427272
INFO : PROGRESS: pass 2, at document #134/166
INFO : merging changes from 67 documents into a model of 166 documents
INFO : topic #2 (0.010): 0.036*"tiet_kiem" + 0.035*"than" + 0.035*"tieng" + 0.025*"lam_viec" + 0.022*"cong_nhan" + 0.019*"nhan" + 0.017*"ke_hoach" + 0.016*"thoi_quen" + 0.015*"tu_duy" + 0.014*"ky_nang"
INFO : topic #55 (0.010): 0.095*"ngon_ngu" + 0.068*"ung_dung" + 0.059*"lap_trinh" + 0.031*"pho_bien" + 0.021*"hoc" + 0.020*"javascript" + 0.018*"java" + 0.018*"toc_đo" + 0.016*"chuong_trinh" + 0.016*"lap_trinh_vien"
INFO : topic #36 (0.010): 0.041*"cuoc_song" + 0.035*"song" + 0.018*"cong_viec" + 0.013*"cang_thang" + 0.012*"y_nghia" + 0.012*"thuong_xuyen" + 0.012*"thoi_quen" + 0.010*"loi_ich" + 0.010*"lanh_manh" + 0.010*"suc_khoe"
INFO : topic #18 (0.010): 0.002

INFO : topic #0 (0.010): 0.201*"user" + 0.165*"stories" + 0.115*"nhac_nho" + 0.044*"marketer" + 0.037*"liet_ke" + 0.028*"good" + 0.027*"me" + 0.020*"cot" + 0.016*"chim" + 0.012*"role"
INFO : topic diff=1.595915, rho=0.365695
INFO : -36.811 per-word bound, 120547374331.9 perplexity estimate based on a held-out corpus of 32 documents with 15007 words
INFO : PROGRESS: pass 4, at document #166/166
INFO : merging changes from 32 documents into a model of 166 documents
INFO : topic #11 (0.010): 0.000*"agile" + 0.000*"ta" + 0.000*"moi_truong" + 0.000*"du_an" + 0.000*"thong_minh" + 0.000*"khach_hang" + 0.000*"quan_ly" + 0.000*"da" + 0.000*"he_thong" + 0.000*"bao_ve"
INFO : topic #26 (0.010): 0.000*"kinh_te" + 0.000*"khung_hoang" + 0.000*"thong_minh" + 0.000*"du_lich" + 0.000*"ve" + 0.000*"calo" + 0.000*"thuc_pham" + 0.000*"the_gioi" + 0.000*"lua_chon" + 0.000*"suy_thoai"
INFO : topic #7 (0.010): 0.037*"phong" + 0.032*"thue" + 0.031*"tro" + 0.021*"van_hoa" + 0.018*"xe" + 0.017*"gia" + 0.015*"ba

INFO : topic #35 (0.010): 0.122*"tien" + 0.079*"đien" + 0.072*"chi_tieu" + 0.041*"thu_nhap" + 0.031*"ve_sinh" + 0.026*"an_ninh" + 0.025*"hop_ly" + 0.024*"app" + 0.022*"tuan" + 0.022*"gia_ca"
INFO : topic diff=0.420860, rho=0.324826
INFO : PROGRESS: pass 7, at document #67/166
INFO : merging changes from 67 documents into a model of 166 documents
INFO : topic #44 (0.010): 0.000*"khoi_nghiep" + 0.000*"moi_truong" + 0.000*"nam" + 0.000*"thi_truong" + 0.000*"kinh_doanh" + 0.000*"kinh_te" + 0.000*"doanh_nghiep" + 0.000*"o_nhiem" + 0.000*"tien" + 0.000*"nguyen"
INFO : topic #47 (0.010): 0.000*"moi_truong" + 0.000*"o_nhiem" + 0.000*"bao_ve" + 0.000*"kinh_te" + 0.000*"he_thong" + 0.000*"khu" + 0.000*"thong_minh" + 0.000*"khung_hoang" + 0.000*"con_nguoi" + 0.000*"y_thuc"
INFO : topic #40 (0.010): 0.720*"tay" + 0.164*"mu" + 0.000*"da" + 0.000*"du_lich" + 0.000*"duong" + 0.000*"mat" + 0.000*"chet" + 0.000*"lan" + 0.000*"kem" + 0.000*"san_pham"
INFO : topic #29 (0.010): 0.120*"ta" + 0.110*"co" + 0

INFO : topic diff=0.265318, rho=0.283096
INFO : PROGRESS: pass 9, at document #134/166
INFO : merging changes from 67 documents into a model of 166 documents
INFO : topic #25 (0.010): 0.026*"bat_đong_san" + 0.026*"khoa_hoc" + 0.023*"ung_dung" + 0.019*"hop_đong" + 0.018*"thi_truong" + 0.016*"doanh_nghiep" + 0.016*"chuyen_gia" + 0.015*"tin_dung" + 0.015*"so" + 0.014*"anh"
INFO : topic #36 (0.010): 0.049*"cuoc_song" + 0.048*"song" + 0.018*"cong_viec" + 0.016*"y_nghia" + 0.015*"huong" + 0.015*"thuong_xuyen" + 0.014*"loi" + 0.012*"tich_cuc" + 0.011*"cang_thang" + 0.011*"lo_lang"
INFO : topic #52 (0.010): 0.269*"calo" + 0.118*"can" + 0.074*"co_the" + 0.041*"nap" + 0.032*"protein" + 0.028*"kcal" + 0.026*"nang_luong" + 0.023*"calories" + 0.022*"van_đong" + 0.021*"đot"
INFO : topic #33 (0.010): 0.076*"mat_ngu" + 0.022*"am" + 0.020*"chua" + 0.015*"nguyen_nhan" + 0.014*"chong" + 0.011*"sau" + 0.011*"hai" + 0.010*"tam" + 0.009*"anh_sang" + 0.009*"che_đo"
INFO : topic #24 (0.010): 0.092*"đau_tu" + 

In [10]:
sentences = make_sentences()
sentences = make_texts_corpus(sentences)
coherence_model_lda = CoherenceModel(model=lda_model, texts=sentences, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

INFO : using ParallelWordOccurrenceAccumulator<processes=7, batch_size=64> to estimate probabilities from sliding windows
INFO : 1 batches submitted to accumulate stats from 64 documents (25378 virtual)
INFO : 2 batches submitted to accumulate stats from 128 documents (49615 virtual)
INFO : 3 batches submitted to accumulate stats from 192 documents (72920 virtual)
INFO : 7 accumulators retrieved from output queue
INFO : accumulated word occurrence stats for 72973 virtual documents



Coherence Score:  0.49037017168042835


In [11]:
import gensim
corpus = gensim.corpora.MmCorpus('../models/corpus.mm')
print('\nPerplexity: ', lda_model.log_perplexity(corpus))

INFO : loaded corpus index from ../models/corpus.mm.index
INFO : initializing cython corpus reader from ../models/corpus.mm
INFO : accepted corpus with 166 documents, 4138 features, 35081 non-zero entries
INFO : -20.993 per-word bound, 2086841.5 perplexity estimate based on a held-out corpus of 166 documents with 78387 words



Perplexity:  -20.992889596594793


In [12]:
def evaluate_quality_model(num_topics, chunksize):
    lda_model = gensim.models.ldamodel.LdaModel(corpus, id2word=id2word, num_topics=num_topics, passes=10,
            chunksize=chunksize, random_state=46, alpha=0.01, eta=0.005,
            minimum_probability=0.0, per_word_topics=False)
    sentences = make_sentences()
    sentences = make_texts_corpus(sentences)
    coherence_model_lda = CoherenceModel(model=lda_model, texts=sentences, dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    perplexity = lda_model.log_perplexity(corpus)
    return coherence_lda, perplexity

In [13]:
list = []
for num_topics in range(55, 90):
    coherence_lda, perplexity = evaluate_quality_model(num_topics, 67)
    list.append((num_topics, coherence_lda, perplexity))

INFO : using serial LDA version on this node
INFO : running online (multi-pass) LDA training, 55 topics, 10 passes over the supplied corpus of 166 documents, updating model once every 67 documents, evaluating perplexity every 166 documents, iterating 50x with a convergence threshold of 0.001000
INFO : PROGRESS: pass 0, at document #67/166
INFO : merging changes from 67 documents into a model of 166 documents
INFO : topic #21 (0.010): 0.035*"kinh_te" + 0.028*"khung_hoang" + 0.016*"thi_truong" + 0.011*"the_gioi" + 0.010*"tai_chinh" + 0.009*"da" + 0.008*"no" + 0.007*"ukraina" + 0.006*"suy_thoai" + 0.006*"nam"
INFO : topic #3 (0.010): 0.032*"calo" + 0.017*"moi_truong" + 0.014*"can" + 0.013*"thuc_pham" + 0.012*"material" + 0.012*"co_the" + 0.009*"design" + 0.009*"o_nhiem" + 0.008*"cung_cap" + 0.008*"thiet_ke"
INFO : topic #17 (0.010): 0.058*"da" + 0.018*"marketing" + 0.015*"lan" + 0.011*"san_pham" + 0.011*"sprint" + 0.010*"duong" + 0.008*"cham_soc" + 0.007*"mat" + 0.007*"lanh_manh" + 0.007*

INFO : topic #17 (0.010): 0.162*"marketing" + 0.143*"sprint" + 0.053*"meeting" + 0.048*"story" + 0.040*"scrum" + 0.039*"user" + 0.037*"review" + 0.037*"mua" + 0.029*"master" + 0.027*"planning"
INFO : topic diff=8.413518, rho=0.427272
INFO : PROGRESS: pass 2, at document #134/166
INFO : merging changes from 67 documents into a model of 166 documents
INFO : topic #10 (0.010): 0.003*"ta" + 0.002*"trang_web" + 0.002*"du_lich" + 0.001*"xay_dung" + 0.001*"chuyen" + 0.001*"api" + 0.001*"thiet_ke" + 0.001*"thong_minh" + 0.001*"mua" + 0.001*"khoi_nghiep"
INFO : topic #9 (0.010): 0.223*"da" + 0.046*"skincare" + 0.042*"lan" + 0.041*"cham_soc" + 0.038*"duong" + 0.035*"mat" + 0.029*"kem" + 0.023*"am" + 0.020*"rua" + 0.018*"quy_trinh"
INFO : topic #5 (0.010): 0.038*"ngu" + 0.027*"song" + 0.020*"đien" + 0.017*"tieng" + 0.016*"xanh" + 0.016*"loi" + 0.016*"than" + 0.015*"soi" + 0.015*"muoi" + 0.011*"hien_đai"
INFO : topic #11 (0.010): 0.107*"du_lieu" + 0.074*"đe_cap" + 0.047*"api" + 0.041*"phep" + 0.03

INFO : topic diff=1.698199, rho=0.365695
INFO : -36.973 per-word bound, 134905865426.7 perplexity estimate based on a held-out corpus of 32 documents with 15007 words
INFO : PROGRESS: pass 4, at document #166/166
INFO : merging changes from 32 documents into a model of 166 documents
INFO : topic #23 (0.010): 0.111*"nha" + 0.096*"user" + 0.042*"thu_muc" + 0.034*"userid" + 0.025*"tuong_tu" + 0.024*"restful" + 0.023*"mongodb" + 0.023*"document" + 0.023*"blog" + 0.022*"nhat"
INFO : topic #28 (0.010): 0.043*"san_xuat" + 0.036*"he_thong" + 0.034*"cong_nghiep" + 0.032*"cong_nghe" + 0.028*"quan_ly" + 0.022*"doanh_nghiep" + 0.018*"nhiem_vu" + 0.017*"nganh" + 0.016*"nang" + 0.014*"hoa"
INFO : topic #36 (0.010): 0.062*"vr" + 0.034*"ao" + 0.025*"ung_dung" + 0.025*"thiet_bi" + 0.019*"trai" + 0.018*"nghiem" + 0.013*"the_gioi" + 0.011*"nen_tang" + 0.010*"cong_nghe" + 0.009*"toc_đo"
INFO : topic #45 (0.010): 0.043*"moi_truong" + 0.030*"xanh" + 0.024*"sinh_thai" + 0.019*"ben_vung" + 0.019*"the_gioi" + 

INFO : topic diff=0.463656, rho=0.324826
INFO : PROGRESS: pass 7, at document #67/166
INFO : merging changes from 67 documents into a model of 166 documents
INFO : topic #22 (0.010): 0.569*"tay" + 0.327*"chet" + 0.015*"duong" + 0.001*"da" + 0.000*"lan" + 0.000*"kho" + 0.000*"san_pham" + 0.000*"mat" + 0.000*"cham_soc" + 0.000*"sach"
INFO : topic #46 (0.010): 0.045*"cong_nghe" + 0.044*"nhan_tao" + 0.040*"thong_minh" + 0.037*"tri_tue" + 0.035*"con_nguoi" + 0.021*"kha_nang" + 0.017*"may" + 0.016*"he_thong" + 0.013*"nghien_cuu" + 0.012*"phan_cung"
INFO : topic #26 (0.010): 0.104*"thanh_than" + 0.062*"quyt" + 0.062*"buoi" + 0.000*"kinh_te" + 0.000*"khung_hoang" + 0.000*"chu_ky" + 0.000*"calo" + 0.000*"thuc_pham" + 0.000*"suy_thoai" + 0.000*"ve"
INFO : topic #27 (0.010): 0.041*"khach_hang" + 0.026*"quyet_đinh" + 0.020*"doanh_nghiep" + 0.019*"san_pham" + 0.014*"cung_cap" + 0.013*"team" + 0.012*"phan_mem" + 0.012*"muc_tieu" + 0.010*"xac_đinh" + 0.010*"tiep_can"
INFO : topic #6 (0.010): 0.156*"b

INFO : topic diff=0.283236, rho=0.283096
INFO : PROGRESS: pass 9, at document #134/166
INFO : merging changes from 67 documents into a model of 166 documents
INFO : topic #36 (0.010): 0.037*"vr" + 0.028*"ung_dung" + 0.025*"thiet_bi" + 0.022*"ao" + 0.018*"trai" + 0.017*"nghiem" + 0.012*"nen_tang" + 0.012*"the_gioi" + 0.011*"pho_bien" + 0.010*"tich_hop"
INFO : topic #26 (0.010): 0.099*"thanh_than" + 0.070*"quyt" + 0.070*"buoi" + 0.000*"kinh_te" + 0.000*"khung_hoang" + 0.000*"chu_ky" + 0.000*"calo" + 0.000*"thuc_pham" + 0.000*"suy_thoai" + 0.000*"ve"
INFO : topic #6 (0.010): 0.158*"bat_đong_san" + 0.148*"tang_truong" + 0.100*"du_bao" + 0.099*"thi_truong" + 0.066*"lai_suat" + 0.050*"suy_thoai" + 0.043*"rui_ro" + 0.043*"lam_phat" + 0.038*"gdp" + 0.035*"my"
INFO : topic #1 (0.010): 0.250*"tiet_kiem" + 0.191*"tien" + 0.070*"khoan" + 0.037*"ngan_hang" + 0.036*"cong_nhan" + 0.030*"can_thiet" + 0.020*"gia_đinh" + 0.020*"chi_phi" + 0.020*"chi" + 0.020*"gui"
INFO : topic #29 (0.010): 0.118*"phu_nu

...

INFO : topic #11 (0.010): 0.022*"agile" + 0.022*"moi_truong" + 0.012*"du_an" + 0.012*"bao_ve" + 0.011*"quan_ly" + 0.010*"da" + 0.008*"khach_hang" + 0.008*"o_nhiem" + 0.007*"team" + 0.006*"san_pham"
INFO : topic #50 (0.010): 0.057*"agile" + 0.024*"du_an" + 0.023*"đoi" + 0.015*"team" + 0.014*"san_pham" + 0.013*"vong" + 0.011*"khach_hang" + 0.010*"phan_mem" + 0.009*"muc_tieu" + 0.009*"scrum"
INFO : topic #0 (0.010): 0.052*"moi_truong" + 0.025*"user" + 0.020*"o_nhiem" + 0.018*"bao_ve" + 0.010*"marketing" + 0.008*"kinh_te" + 0.008*"stories" + 0.006*"noi_dung" + 0.006*"khach_hang" + 0.006*"khu"
INFO : topic diff=120.406158, rho=1.000000
INFO : PROGRESS: pass 0, at document #134/166
INFO : merging changes from 67 documents into a model of 166 documents
INFO : topic #11 (0.010): 0.018*"agile" + 0.018*"moi_truong" + 0.010*"du_an" + 0.010*"bao_ve" + 0.009*"quan_ly" + 0.008*"da" + 0.007*"khach_hang" + 0.006*"o_nhiem" + 0.006*"team" + 0.005*"san_pham"
INFO : topic #57 (0.010): 0.039*"luu_luong" + 

INFO : topic #87 (0.010): 0.004*"tien" + 0.003*"da" + 0.002*"khoi_nghiep" + 0.002*"đau_tu" + 0.002*"moi_truong" + 0.001*"gia" + 0.001*"thi_truong" + 0.001*"mua" + 0.001*"ao" + 0.001*"the_gioi"
INFO : topic #88 (0.010): 0.005*"calo" + 0.004*"da" + 0.003*"can" + 0.002*"san_pham" + 0.002*"agile" + 0.002*"uong" + 0.002*"thuc_pham" + 0.002*"sprint" + 0.002*"ao" + 0.001*"đau_tu"
INFO : topic #1 (0.010): 0.166*"tien" + 0.060*"tiet_kiem" + 0.047*"khoan" + 0.032*"ngan_hang" + 0.020*"chi" + 0.019*"đong" + 0.018*"lai_suat" + 0.017*"luong" + 0.017*"tai_khoan" + 0.016*"tuan"
INFO : topic diff=5.743732, rho=0.427272
INFO : -37.256 per-word bound, 164161658822.2 perplexity estimate based on a held-out corpus of 32 documents with 15007 words
INFO : PROGRESS: pass 2, at document #166/166
INFO : merging changes from 32 documents into a model of 166 documents
INFO : topic #17 (0.010): 0.011*"da" + 0.004*"lan" + 0.002*"san_pham" + 0.002*"duong" + 0.002*"cham_soc" + 0.002*"mat" + 0.002*"lanh_manh" + 0.001*

INFO : topic #64 (0.010): 0.171*"ao" + 0.049*"ta" + 0.038*"co" + 0.033*"hai" + 0.026*"nam" + 0.023*"tri" + 0.023*"phan_cung" + 0.020*"trang_phuc" + 0.019*"truyen_thong" + 0.018*"virtual"
INFO : topic #63 (0.010): 0.195*"react" + 0.081*"online" + 0.059*"may" + 0.056*"thiet_ke" + 0.054*"components" + 0.044*"mui" + 0.025*"hien_đai" + 0.024*"ok" + 0.019*"cua" + 0.018*"đac_điem"
INFO : topic #35 (0.010): 0.000*"kinh_te" + 0.000*"moi_truong" + 0.000*"thi_truong" + 0.000*"mua" + 0.000*"o_nhiem" + 0.000*"bat_đong_san" + 0.000*"bao_ve" + 0.000*"khung_hoang" + 0.000*"đau_tu" + 0.000*"nhu_cau"
INFO : topic #4 (0.010): 0.001*"da" + 0.000*"moi_truong" + 0.000*"kinh_doanh" + 0.000*"khoi_nghiep" + 0.000*"san_pham" + 0.000*"quan_ly" + 0.000*"bao_ve" + 0.000*"song" + 0.000*"đau_tu" + 0.000*"dau"
INFO : topic diff=0.847094, rho=0.365695
INFO : PROGRESS: pass 5, at document #67/166
INFO : merging changes from 67 documents into a model of 166 documents
INFO : topic #21 (0.010): 0.000*"kinh_te" + 0.000*"un

INFO : topic #45 (0.010): 0.149*"moi_truong" + 0.068*"xanh" + 0.039*"o_nhiem" + 0.038*"bao_ve" + 0.029*"song" + 0.028*"tai_nguyen" + 0.024*"sinh_thai" + 0.024*"khi_hau" + 0.023*"ben_vung" + 0.017*"bien_đoi"
INFO : topic #42 (0.010): 0.000*"da" + 0.000*"lan" + 0.000*"mat" + 0.000*"sach" + 0.000*"song" + 0.000*"cham_soc" + 0.000*"suc_khoe" + 0.000*"san_pham" + 0.000*"kinh_doanh" + 0.000*"kem"
INFO : topic #80 (0.010): 0.291*"da" + 0.059*"skincare" + 0.057*"cham_soc" + 0.047*"lan" + 0.039*"duong" + 0.038*"mat" + 0.035*"kem" + 0.031*"san_pham" + 0.025*"dau" + 0.024*"rua"
INFO : topic #55 (0.010): 0.226*"ngon_ngu" + 0.139*"lap_trinh" + 0.054*"java" + 0.040*"ung_dung" + 0.035*"php" + 0.034*"javascript" + 0.032*"swift" + 0.030*"pho_bien" + 0.025*"python" + 0.025*"web"
INFO : topic diff=0.251886, rho=0.308936
INFO : PROGRESS: pass 7, at document #134/166
INFO : merging changes from 67 documents into a model of 166 documents
INFO : topic #88 (0.010): 0.000*"calo" + 0.000*"da" + 0.000*"can" + 0.

INFO : topic #76 (0.010): 0.000*"kinh_te" + 0.000*"du_bao" + 0.000*"imf" + 0.000*"ngan_hang" + 0.000*"suy_thoai" + 0.000*"the_gioi" + 0.000*"lai_suat" + 0.000*"tang_truong" + 0.000*"chat" + 0.000*"my"
INFO : topic #23 (0.010): 0.229*"tra" + 0.205*"chua" + 0.115*"nu" + 0.111*"mat_ong" + 0.106*"ngon" + 0.094*"thuoc" + 0.039*"nguyen_lieu" + 0.008*"thieu_hut" + 0.006*"chung_benh" + 0.005*"lua_mi"
INFO : topic #16 (0.010): 0.127*"vr" + 0.062*"ao" + 0.060*"thiet_bi" + 0.038*"thuc" + 0.033*"trai" + 0.032*"cong_nghe" + 0.032*"the_gioi" + 0.027*"tich_hop" + 0.025*"nen_tang" + 0.021*"san_pham"
INFO : topic #83 (0.010): 0.000*"kinh_te" + 0.000*"khung_hoang" + 0.000*"da" + 0.000*"thi_truong" + 0.000*"lan" + 0.000*"bat_đong_san" + 0.000*"doanh_nghiep" + 0.000*"gia" + 0.000*"tai_chinh" + 0.000*"suy_thoai"
INFO : topic diff=0.162875, rho=0.283096
INFO : -37.060 per-word bound, 143277773878.1 perplexity estimate based on a held-out corpus of 32 documents with 15007 words
INFO : PROGRESS: pass 9, at do

In [14]:
list

[(55, 0.4831244138222541, -21.155758114258585),
 (56, 0.48650014545076176, -21.286874886046462),
 (57, 0.49037017168042835, -20.992889596594793),
 (58, 0.5037433768927638, -21.131066860084697),
 (59, 0.49255165109218046, -21.15049395980862),
 (60, 0.4642161772341318, -21.108278405948912),
 (61, 0.48849646644953465, -21.16533373122797),
 (62, 0.48165313257377007, -21.19084390301028),
 (63, 0.48615931625921005, -21.056190064200464),
 (64, 0.4722554882771848, -21.175339502277986),
 (65, 0.4990978625595965, -21.26608702259624),
 (66, 0.4978229308582385, -21.176947060263625),
 (67, 0.48779306263339794, -21.190827028487245),
 (68, 0.4743133602400225, -21.167482881939325),
 (69, 0.5194053426687818, -21.202916920601023),
 (70, 0.4704906986713356, -21.184145607910526),
 (71, 0.47054442171100647, -21.225795837832397),
 (72, 0.4720817143608088, -21.194660257001708),
 (73, 0.46063703443898485, -21.301359988463215),
 (74, 0.4717335459270573, -21.18879717958202),
 (75, 0.4520889547122301, -21.113285